In [8]:
!pip install openpyxl

     ------------------------------------ 242.1/242.1 kB 646.0 kB/s eta 0:00:00


In [1]:
import os
import pandas as pd
import json
import pickle
import urllib.request
import glob
from PIL import Image, ImageSequence
import img2pdf
import numpy as np
import shutil
import zipfile

In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import requests

PATH = 'C:\Program Files (x86)\chromedriver.exe'
DOWNLOAD_PATH = 'C:/Users/ebryaga/Downloads'

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
driver = webdriver.Chrome(PATH)

driver.get('https://pmhd.azure.bakerhughes.com/pmhd2/html/searchEvent-op-ses')

## Action: Now sign in in pop up window/ Click Agree

In [5]:
def search_and_download(x):
    '''
    This function enables you to serach event by event ID and downaload attachments.
    (Downloading using big green arrow on result page)
    
    Args:
        x - event ID
        
    Returns:
        Download file.
    '''
    
    print(f"Downloading {x}")
    driver.get('https://pmhd.azure.bakerhughes.com/pmhd2/html/searchEvent-op-ses')
    search = driver.find_element("name", "eventId")
    search.clear()
    search.send_keys(x)
    
   
    submit = driver.find_element('xpath', '//*[@id="gbl_mainBody"]/form/table[2]/tbody/tr[2]/td[1]/input[1]')
    submit.click()
    
    download = driver.find_element('xpath', '//*[@id="tbl_events"]/tbody/tr/td[12]/a')
    download.click()
    
    
    
    #switch to search
    driver.get('https://pmhd.azure.bakerhughes.com/pmhd2/html/searchEvent-op-ses')

In [6]:
def move_and_unzip():
    '''
    This function enables you to move downloaded zip file to working directory folder (defined by os.getcwd()) and uznip it.
    
    Args:
        
    Returns:
        Move and uznip file in 
    '''
    
    # Waiting till zip will be download
    downloaded = False
    while not downloaded:
        try:
            list_of_files = glob.glob(f'{DOWNLOAD_PATH}/*')
            latest_file = max(list_of_files, key=os.path.getctime)
            norm_latest_file = str(os.path.normpath(latest_file))
            downloaded = True
        except FileNotFoundError:
            time.sleep(2)
        while (norm_latest_file.split('.')[-1] != 'zip'):
            time.sleep(2)
            list_of_files = glob.glob(f'{DOWNLOAD_PATH}/*') # * means all if need specific format then *.csv
            try:
                latest_file = max(list_of_files, key=os.path.getctime)
                norm_latest_file = str(os.path.normpath(latest_file))
                print(norm_latest_file)
                if norm_latest_file.split('.')[-1] == 'zip':
                        break
            except FileNotFoundError:
                time.sleep(1)

                    
    # Move zip file to specified direction

    src = norm_latest_file
    file = norm_latest_file.split('\\')[-1]
    dst = f'{os.getcwd()}\\{file}'
    shutil.move(src,dst)    
    
    
    with zipfile.ZipFile(file, "r") as zip_ref:
        zip_ref.extractall(f"{file.split('.')[0]}")

## Preparing input - list  of events

In [9]:
df_pmhd = pd.read_excel('pmhd_K_final.xlsx')
events = df_pmhd['N_EVT_ID'].to_list()

In [10]:
os.chdir('C:\\Users\\ebryaga\\Desktop\\PMHD_Test')

In [11]:
bad_events = []
for event in events[:4]:
    try:
        search_and_download(event)
        move_and_unzip()
        print("OK")
    except:
        bad_events.append(event)
        print("Whoops")

OK
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip
OK
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip.crdownload
C:\Users\ebryaga\Downloads\Event_223129.zip
OK
OK
